In [15]:
import os
import torch
from PIL import Image
import timm
from torchvision import transforms
import glob

# 画像の前処理: リサイズ、テンソル変換、正規化
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 学習済みモデルのロード: ViT モデルを事前学習済みの重みで初期化
model = timm.create_model("vit_base_patch16_224", pretrained=True)
# M1/M2 macのGPUを利用する設定
mps_device = torch.device("mps")
# 学習済みモデルをGPUに配置
model.to(mps_device)
# モデルを（評価）推論モードに設定
model.eval()

# 推論（今回は画像認識）対象となる画像の設定（dataディレクトリの中にあるすべての画像を対象にする）
image_paths = glob.glob("data/*.jpg")

# クラスラベルを読み込む: image_classes.txt からクラスラベルのリストを作成
with open('image_classes.txt', 'r') as f:
    class_labels = [line.strip() for line in f.readlines()]

# dataディレクトリに格納された各画像に対して推論を実行
for image_path in image_paths:
    # 画像を読み込み、RGB に変換
    image = Image.open(image_path).convert("RGB")
    # モデルに入力できる形（テンソル）に変換
    image_tensor = transform(image).unsqueeze(0).to(mps_device)
    
    with torch.no_grad():
        output = model(image_tensor)
        probabilities = torch.nn.functional.softmax(output, dim=1)

    # 結果を表示: 認識結果の確率トップ5を表示
    topk_probs, topk_indices = torch.topk(probabilities, k=5)
    topk_probs = topk_probs.squeeze().tolist()
    topk_indices = topk_indices.squeeze().tolist()
    print("Top-5 Predictions:")
    for i in range(len(topk_probs)):
        class_name = class_labels[topk_indices[i]]
        prob = topk_probs[i] * 100
        print(f"{i+1}. {class_name} ({prob:.2f}%)")


Top-5 Predictions:
1. 285: 'Egyptian cat', (55.46%)
2. 282: 'tiger cat', (12.03%)
3. 281: 'tabby, tabby cat', (11.34%)
4. 287: 'lynx, catamount', (3.64%)
5. 283: 'Persian cat', (3.27%)
Top-5 Predictions:
1. 209: 'Chesapeake Bay retriever', (93.90%)
2. 208: 'Labrador retriever', (1.00%)
3. 180: 'American Staffordshire terrier, Staffordshire terrier, American pit bull terrier, pit bull terrier', (0.86%)
4. 178: 'Weimaraner', (0.76%)
5. 206: 'curly-coated retriever', (0.70%)
